# Talking Therapies for Anxiety and Depression dataset

In [1]:
import sys
import os
script_fp = "../../../../scripts/"
sys.path.insert(0, script_fp)
from data_doc_helper import DocHelper
import mdapi_functions as md
API_KEY = os.environ['FASTAPI_KEY']
import requests
import json
import pandas as pd
import datacite_api_functions as dcf
from IPython.display import display, Markdown

In [4]:
md.get_nhse_cohort_counts("DEMOGRAPHICS")

,cohort,count
0,ALSPAC,6256
1,BCS70,6482
2,BIB,28427
3,ELSA,7102
4,EPICN,15117
5,EXCEED,9504
6,FENLAND,10595
7,GENSCOT,0
8,GLAD,67123
9,MCS,21907


In [ ]:
# code to get number of tables and number of variables for AB talk - 28/05/25
dsvs = md.get_md_api_dsvs()
dsvs["version_num"] = dsvs["version_num"].fillna("v0001")
dsvs["vnum"] = dsvs["version_num"].apply(lambda x: int(str(x).split("v")[1]))
dsvs2 = dsvs.sort_values(by="vnum", ascending=False).drop_duplicates(subset="table")
dsvs2["source_table"] = dsvs["source"] + "_" + dsvs["table"]
print("Tables: {} , Variables: {}".format(len(dsvs), int(dsvs["num_columns"].sum())))
print("Tables: {} , Variables: {}".format(len(dsvs2), int(dsvs2["num_columns"].sum())))

Tables: 680 , Variables: 254290
Tables: 428 , Variables: 156830


In [ ]:
# part of get_descriptor_table
def get_nhse_ds(x):
    ds = md.get_md_api_dss()
    df_ds = ds[(ds["source"] == "NHSE") & (ds["table"] == x)]
    df_ds["source_table"] = df_ds["source"] + "_" + df_ds["table"]
    ss = md.get_md_api_ss()[["source", "Owner"]]
    df_ds = df_ds.merge(ss, on="source")
    return df_ds

get_nhse_ds("IAPT")

C:\Users\sn9395\AppData\Local\Temp\ipykernel_9636\1443449575.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ds["source_table"] = df_ds["source"] + "_" + df_ds["table"]


,source,index,table_name,collection_start,participants_invited,links,special_conditions,source_name,table,short_desc,long_desc,collection_time,collection_end,participants_included,topic_tags,source_type,source_table,Owner
0,NHSE,379,Improving Access to Psychological Therapies (I...,04/2012,None,[digital.nhs.uk](https://digital.nhs.uk/data-a...,None,NHS England,IAPT,People treated in England for depression and a...,This dataset will shortly be renamed the NHS T...,2012-04-01 00:00:00,ongoing,26601,None,Linked,NHSE_IAPT,"NHS England, Office for National Statistics an..."


In [12]:
df = md.get_md_api_dsvs()
df = df[(df["source"] == "nhsd") & (df["table"].str.startswith("IAPT"))].drop_duplicates(subset="table")
df

,source,values_table,index,description_table,version_date,num_rows,table_full,table,version_num,num_columns,num_participants
550,nhsd,None,550,IAPT_care_activities_v0002_description,NaN,508758.0,IAPT_care_activities_v0002,IAPT_care_activities,v0002,47.0,NaN
552,nhsd,None,552,IAPT_care_cluster_v0002_description,NaN,45723.0,IAPT_care_cluster_v0002,IAPT_care_cluster,v0002,12.0,NaN
554,nhsd,None,554,IAPT_coded_scored_assessments_v0002_description,NaN,1267393.0,IAPT_coded_scored_assessments_v0002,IAPT_coded_scored_assessments,v0002,17.0,NaN
556,nhsd,None,556,IAPT_demographics_and_referral_v0002_description,NaN,58680.0,IAPT_demographics_and_referral_v0002,IAPT_demographics_and_referral,v0002,192.0,NaN
558,nhsd,None,558,IAPT_employment_status_v0002_description,NaN,115830.0,IAPT_employment_status_v0002,IAPT_employment_status,v0002,21.0,NaN
560,nhsd,None,560,IAPT_internet_enabled_therapies_v0002_description,NaN,18057.0,IAPT_internet_enabled_therapies_v0002,IAPT_internet_enabled_therapies,v0002,18.0,NaN
562,nhsd,None,562,IAPT_mental_and_physical_health_conditions_v00...,NaN,79715.0,IAPT_mental_and_physical_health_conditions_v0002,IAPT_mental_and_physical_health_conditions,v0002,17.0,NaN
564,nhsd,None,564,IAPT_onward_referrals_v0002_description,NaN,1193.0,IAPT_onward_referrals_v0002,IAPT_onward_referrals,v0002,13.0,NaN
566,nhsd,None,566,IAPT_waiting_time_pauses_v0002_description,NaN,1889.0,IAPT_waiting_time_pauses_v0002,IAPT_waiting_time_pauses,v0002,14.0,NaN


In [61]:
url = "https://metadata-api-4a09f2833a54.herokuapp.com/nhs-dataset-counts/?table={}".format(df.iloc[5]["table"])
r = requests.get(url, headers={'access_token': API_KEY})
data = r.text
pj = json.loads(data)
dfcc = pd.json_normalize(pj)[["cohort", "count"]]
dfcc = dfcc[~dfcc['cohort'].isin(['GENSCOT', 'NICOLA', 'SABRE'])]
dfcc["count2"] = dfcc["count"].replace("<10", "0").astype(int)
dfcc.loc[dfcc.index[-1]] = ["TOTAL", int(dfcc['count2'].sum()), int(dfcc['count2'].sum())]
dfcc = dfcc.drop(columns="count2")
print(df.iloc[5]["table"])
dfcc

IAPT_internet_enabled_therapies


,cohort,count
0,ALSPAC,85
1,BCS70,18
2,BIB,36
3,ELSA,<10
4,EPICN,0
5,EXCEED,24
6,FENLAND,<10
8,GLAD,1528
9,MCS,120
10,NCDS58,<10


In [57]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider, Select
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook(hide_banner=True)


In [83]:
tbl_names = []
metrics_tables = []
for i in range(0, len(df)-1):
    url = "https://metadata-api-4a09f2833a54.herokuapp.com/nhs-dataset-counts/?table={}".format(df.iloc[i]["table"])
    r = requests.get(url, headers={'access_token': API_KEY})
    data = r.text
    pj = json.loads(data)
    dfcc = pd.json_normalize(pj)[["cohort", "count"]]
    dfcc = dfcc[~dfcc['cohort'].isin(['GENSCOT', 'NICOLA', 'SABRE'])]
    tbl_names += len(dfcc) * [df.iloc[i]["table"]]
    #tbl_names.append(df.iloc[i]["table"] * len(dfcc))
    dfcc["count"] = dfcc["count"].replace("<10", "0").astype(int)
    metrics_tables.append(dfcc)

In [84]:
tbl_names

['IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_activities',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_care_cluster',
 'IAPT_coded_scored_assessments',
 'IAPT_coded_scored_assessments',
 'IAPT_coded_scored_assessments',
 'IAPT_coded_scored_assessments',
 'IAPT_coded_scored_assessments',
 'IAPT_coded_scored_assessments

In [12]:
from bokeh.io import show
from bokeh.models import (ColumnDataSource, DataCube, GroupingInfo,
                          StringFormatter, SumAggregator, TableColumn)
from bokeh.io import output_notebook
output_notebook(hide_banner=True)

df = md.get_md_api_dsvs()
df = df[(df["source"] == "nhsd") & (df["table"].str.startswith("IAPT"))].drop_duplicates(subset="table")
df

tbl_names = []
metrics_tables = []
for i in range(0, len(df)):
    url = "https://metadata-api-4a09f2833a54.herokuapp.com/nhs-dataset-counts/?table={}".format(df.iloc[i]["table"])
    r = requests.get(url, headers={'access_token': API_KEY})
    data = r.text
    pj = json.loads(data)
    dfcc = pd.json_normalize(pj)[["cohort", "count"]]
    dfcc = dfcc[~dfcc['cohort'].isin(['GENSCOT', 'NICOLA', 'SABRE'])]
    tbl_names += len(dfcc) * [df.iloc[i]["table"]]
    #tbl_names.append(df.iloc[i]["table"] * len(dfcc))
    dfcc["count"] = dfcc["count"].replace("<10", "0").astype(int)
    metrics_tables.append(dfcc)

source = ColumnDataSource(data=dict(
    d0=tbl_names,
    d1=pd.concat(metrics_tables)["cohort"].to_list(),
    px=pd.concat(metrics_tables)["count"].to_list(),
))

target = ColumnDataSource(data=dict(row_indices=[], labels=[]))

formatter = StringFormatter(font_style='bold')

columns = [
    TableColumn(field='d1', title='IAPT Dataset', width=80, sortable=False, formatter=formatter),
    TableColumn(field='px', title='Participant Count', width=40, sortable=False),
]

grouping = [
    GroupingInfo(getter='d0', aggregators=[SumAggregator(field_='px')]),
]

cube = DataCube(source=source, columns=columns, grouping=grouping, target=target)

show(cube)

In [ ]:
from bokeh.io import show
from bokeh.models import (ColumnDataSource, DataCube, GroupingInfo,
                          StringFormatter, SumAggregator, TableColumn)

source = ColumnDataSource(data=dict(
    d0=['A', 'E', 'E', 'E', 'J', 'L', 'M'],
    d1=['B', 'D', 'D', 'H', 'K', 'L', 'N'],
    d2=['C', 'F', 'G', 'H', 'K', 'L', 'O'],
    px=[10, 20, 30, 40, 50, 60, 70],
))

target = ColumnDataSource(data=dict(row_indices=[], labels=[]))

formatter = StringFormatter(font_style='bold')

columns = [
    TableColumn(field='d2', title='Name', width=80, sortable=False, formatter=formatter),
    TableColumn(field='px', title='Price', width=40, sortable=False),
]

grouping = [
    GroupingInfo(getter='d0', aggregators=[SumAggregator(field_='px')]),
    GroupingInfo(getter='d1', aggregators=[SumAggregator(field_='px')]),
]

cube = DataCube(source=source, columns=columns, grouping=grouping, target=target)

show(cube)

In [60]:
x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(width=400, height=400, x_range=(0, 1), y_range=(0, 1))

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    const f = cb_obj.value
    const x = source.data.x
    const y = Array.from(x, (x) => Math.pow(x, f))
    source.data = { x, y }
""")

# slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
# slider.js_on_change('value', callback)

select = Select(title="Option:", value="1", options=["1", "2", "3", "4"])
select.js_on_change('value', callback)

layout = column(select, plot)

show(layout)